This notebook shows all the attributes of ReadoutNetCDF, SweepGroup, and TimestreamGroup objects.

In [1]:
from kid_readout.measurement.io import readoutnc

In [2]:
none_filename = '/data/readout/2015-05-11_150346_compressor_noise_60_s.nc'
bb_filename = '/data/readout/2015-05-12_113832_mmw_noise_broadband.nc'
cw_filename = '/data/readout/2015-05-18_102907_mmw_continuous-wave_140_GHz.nc'
rnc = readoutnc.ReadoutNetCDF(none_filename)

In [3]:
rnc.sweeps

In [4]:
rnc.timestreams

In [5]:
sg = rnc.sweeps[1]
tg = rnc.timestreams[1]

# Dataset

In [6]:
nc = rnc.ncroot
nc

In [7]:
nc.ncattrs()

[u'gitinfo', u'boffile']

In [8]:
nc.variables.keys()

[]

In [9]:
nc.groups.keys()

[u'sweeps', u'timestreams', u'cryo', u'hw_state', u'adc_snaps']

The `cryo` group seems to be empty:

In [10]:
nc.groups['cryo'].ncattrs()

[]

In [11]:
nc.groups['cryo'].variables.keys()

[]

In [12]:
nc.groups['cryo'].groups.keys()

[]

The `adc_snaps` group is empty here too:

In [13]:
nc.groups['adc_snaps'].ncattrs()

[]

These get created in `data_file.DataFile`:

In [14]:
nc.groups['adc_snaps'].variables.keys()

[u'epoch', u'data']

In [15]:
nc.groups['adc_snaps'].variables['epoch'].shape

(0,)

In [16]:
nc.groups['adc_snaps'].variables['data'].shape

(0, 2, 4096)

In [17]:
nc.groups['adc_snaps'].groups.keys()

[]

The `hw_state` group holds the results of `DataFile.log_hw_state()` calls:

In [18]:
nc.groups['hw_state'].ncattrs()

[]

In [19]:
nc.groups['hw_state'].variables.keys()

[u'epoch',
 u'adc_atten',
 u'dac_atten',
 u'ntones',
 u'modulation_rate',
 u'modulation_output']

All of the above have the dimension below:

In [20]:
nc.groups['hw_state'].dimensions.keys()

[u'time']

In [21]:
nc.groups['hw_state'].variables['epoch'][:]

array([  1.43137103e+09,   1.43137129e+09,   1.43137147e+09,
         1.43137156e+09,   1.43137185e+09])

This variable is the total number of tones in the waveform, which may be greater than the number being read out:

In [22]:
nc.groups['hw_state'].variables['ntones'][:]

array([16, 16, 16, 16, 16], dtype=int32)

In [23]:
nc.groups['hw_state'].groups.keys()

[]

# ReadoutNetCDF

In [24]:
rnc.adc_atten

array([ 31.5,  31.5,  31.5,  31.5,  31.5], dtype=float32)

In [25]:
rnc.boffile

u'bb2xpfb14mcr17b_2015_Apr_21_1159.bof'

In [26]:
rnc.dac_atten

array([ 32.,  32.,  32.,  32.,  32.], dtype=float32)

In [27]:
rnc.filename

'/data/readout/2015-05-11_150346_compressor_noise_60_s.nc'

In [28]:
rnc.get_delay_estimate()

-6.33e-05

In [29]:
rnc.get_effective_dac_atten_at(rnc.hardware_state_epoch[0])

(32.0, 54.04119982655925)

In [30]:
rnc.get_modulation_state_at(rnc.hardware_state_epoch[0])

(1, 0)

In [31]:
print(rnc.gitinfo)

commit 5a79f1ef014c88c9f74f4096692434af02fd2855
Author: Daniel Flanigan <daniel.isaiah.flanigan@gmail.com>
Date:   Wed May 6 15:13:28 2015 -0400

    select_bank fix in roach_interface plus various acquisition scripts



In [32]:
rnc.hardware_state_epoch

array([  1.43137103e+09,   1.43137129e+09,   1.43137147e+09,
         1.43137156e+09,   1.43137185e+09])

In [33]:
rnc.heterodyne

False

In [34]:
rnc.mmw_atten_turns

(nan, nan)

In [35]:
rnc.modulation_output

array([1, 1, 1, 1, 1], dtype=int32)

In [36]:
rnc.modulation_rate

array([0, 0, 0, 0, 0], dtype=int32)

In [37]:
rnc.ncroot

In [38]:
rnc.num_tones

array([16, 16, 16, 16, 16], dtype=int32)

# SweepGroup

In [39]:
sg.end_epoch

1431371843.756763

In [40]:
sg.errors.shape

(496,)

In [41]:
sg.frequency.shape

(496,)

In [42]:
sg.index.shape

(496,)

In [43]:
sg.parent

In [44]:
sg.s21.shape

(496,)

In [45]:
sg.start_epoch

1431371820.5151131

In [46]:
sg.timestream_group

In [47]:
stg = sg.timestream_group

Some of arrays in the TimestreamGroup of a SweepGroup are sorted by frequency / tone bin in ascending order:

In [48]:
stg.tonebin.shape

(496,)

In [49]:
np.all(np.diff(stg.tonebin))

True

This array contains `ri.fpga_fft_readout_indexes + 1`, where `ri` is a RoachInterface instance, so it's not monotonic. It also contains duplicates because the fft bins are much wider than the tone bins.

In [50]:
stg.fftbin.shape

(496,)

In [51]:
np.all(np.diff(stg.fftbin))

False

Even when the millimeter-wave source is being used, the TimestreamGroup of a SweepGroup doesn't have the mmw_source_freq, zbd_voltage, or zbd_power attributes:

In [52]:
try:
    print(stg.mmw_source_freq)
except AttributeError:
    print("No mmw_source_freq")

No mmw_source_freq


In [53]:
try:
    print(stg.zbd_voltage)
except AttributeError:
    print("No zbd_voltage")

No zbd_voltage


In [54]:
try:
    print(stg.zbd_power_dbm)
except AttributeError:
    print("No zbd_power_dbm")

No zbd_power_dbm


# TimestreamGroup

In [55]:
tg.adc_sampling_freq

array([ 512.,  512.,  512.,  512.,  512.,  512.,  512.,  512.,  512.,
        512.,  512.,  512.,  512.,  512.,  512.,  512.])

In [56]:
tg.baseband_measurement_freq

array([  96.06689453,  101.96484375,  108.8671875 ,  112.26489258,
        117.27490234,  127.7890625 ,  195.71704102,  188.86694336,
        184.55859375,  178.94555664,  174.95996094,  165.41259766,
        164.37890625,  164.08764648,  148.24975586,  144.19262695])

In [57]:
tg.data.shape

(16, 1048576)

In [58]:
tg.data_len_seconds

array([ 67.108864,  67.108864,  67.108864,  67.108864,  67.108864,
        67.108864,  67.108864,  67.108864,  67.108864,  67.108864,
        67.108864,  67.108864,  67.108864,  67.108864,  67.108864,
        67.108864])

In [59]:
tg.epoch

array([  1.43137185e+09,   1.43137185e+09,   1.43137185e+09,
         1.43137185e+09,   1.43137185e+09,   1.43137185e+09,
         1.43137185e+09,   1.43137185e+09,   1.43137185e+09,
         1.43137185e+09,   1.43137185e+09,   1.43137185e+09,
         1.43137185e+09,   1.43137185e+09,   1.43137185e+09,
         1.43137185e+09])

A TimestreamGroup not associated with a SweepGroup has arrays that are sorted in order of ascending `tg.fftbin`:

In [60]:
tg.fftbin

array([ 6149,  6527,  6969,  7186,  7507,  8179, 12051, 12490, 12765,
       13124, 13380, 13991, 14057, 14075, 15089, 15349], dtype=int32)

In [61]:
tg.get_data_index(0).shape

(1048576,)

The `frequency` and `tonebin` arrays are not monotonic:

In [62]:
tg.measurement_freq

array([  96.06689453,  101.96484375,  108.8671875 ,  112.26489258,
        117.27490234,  127.7890625 ,  195.71704102,  188.86694336,
        184.55859375,  178.94555664,  174.95996094,  165.41259766,
        164.37890625,  164.08764648,  148.24975586,  144.19262695])

In [63]:
tg.mmw_source_freq

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.])

In [64]:
tg.mmw_source_modulation_freq

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.])

In [65]:
tg.modulation_duty_cycle

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.])

In [66]:
tg.modulation_freq

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.])

In [67]:
tg.modulation_period_samples

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.])

In [68]:
tg.modulation_phase

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.])

In [69]:
tg.nfft

array([16384, 16384, 16384, 16384, 16384, 16384, 16384, 16384, 16384,
       16384, 16384, 16384, 16384, 16384, 16384, 16384], dtype=int32)

In [70]:
tg.num_data_samples

1048576

In [71]:
tg.parent

In [72]:
tg.sample_rate

array([ 15625.,  15625.,  15625.,  15625.,  15625.,  15625.,  15625.,
        15625.,  15625.,  15625.,  15625.,  15625.,  15625.,  15625.,
        15625.,  15625.])

In [73]:
tg.sweep_index is None

True

In [74]:
tg.tone_nsamp

array([2097152, 2097152, 2097152, 2097152, 2097152, 2097152, 2097152,
       2097152, 2097152, 2097152, 2097152, 2097152, 2097152, 2097152,
       2097152, 2097152], dtype=int32)

This array is not monotonic:

In [75]:
tg.tonebin

array([393490, 417648, 445920, 459837, 480358, 523424, 801657, 773599,
       755952, 732961, 716636, 677530, 673296, 672103, 607231, 590613], dtype=int32)

In [76]:
tg.wavenorm

array([  1.28636312e-05,   1.28636312e-05,   1.28636312e-05,
         1.28636312e-05,   1.28636312e-05,   1.28636312e-05,
         1.28636312e-05,   1.28636312e-05,   1.28636312e-05,
         1.28636312e-05,   1.28636312e-05,   1.28636312e-05,
         1.28636312e-05,   1.28636312e-05,   1.28636312e-05,
         1.28636312e-05])

In [77]:
tg.zbd_power_dbm

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.])

In [78]:
tg.zbd_voltage

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.])